In [1]:
import tensorflow as tf

/is/sg/klee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### l1_regularization

In [3]:
def l1_reg(A):
    '''
    Assume A is of the form: n-by-d
    '''
    tmp = tf.abs(A)
    return tf.reduce_mean(tf.reduce_sum(tmp, axis=1))

In [4]:
# Testing the l1_reg
A = tf.constant([[1, -2],
                [-4, -3]])
l1_reg(A)
with tf.Session() as sess:
    print(sess.run(A))
    print(sess.run(tf.abs(A)))
    print(sess.run(l1_reg(A)))

[[ 1 -2]
 [-4 -3]]
[[1 2]
 [4 3]]
5


### Jacobian w.r.t The Input

In [5]:
def J_x(H,x):
    J = []
    for i in range(H.shape[1]):
        J_j = tf.gradients(H[:,i],x)
        J.append(J_j[0])
    return tf.stack(J, axis=1)

In [16]:
# A very simple model
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[None,2])
A = tf.constant([[1, -2],
                [-4, -3]], dtype=tf.float32)
h = tf.matmul(x, A)


B = J_x(h, x)
F = tf.norm(B)
F1 = tf.norm(B, axis=[1,2]) # taking the norm over the two later parts
reg_cae = tf.reduce_mean(F1)
print(h.shape)
print(B.shape)
print('F1: {}'.format(F1.shape))
print('reg_cae: {}'.format(reg_cae.shape))

(?, 2)
(?, 2, 2)
F1: (?,)
reg_cae: ()


In [18]:
with tf.Session() as sess:        
    # Note that. d/dx (xA) = A^t
    tmp1 = sess.run(B, feed_dict={x: [[1,3], [2,3]]})
    tmp2 = sess.run(B, feed_dict={x: [[1,3]]})
    norm_1 = sess.run(F1, feed_dict={x: [[1,3], [1,3]]})
    reg_loss = sess.run(reg_cae, feed_dict={x: [[1,3], [1,3]]})
    print(tmp1)
    print(tmp2)
    print(norm_1)
    print(reg_loss)

[[[ 1. -4.]
  [-2. -3.]]

 [[ 1. -4.]
  [-2. -3.]]]
[[[ 1. -4.]
  [-2. -3.]]]
[5.477226 5.477226]
5.477226


### Norm evaluation